# CONFIGURATION

In [6]:
import os
import pandas as pd
from openai import OpenAI
import time
import json
import numpy as np

# --- Configuration ---
CSV_FOLDER = 'videos4/projfiles'
CSV_FILE_MUSIC = os.path.join(CSV_FOLDER, 'input', 'youtube_comments_audio.csv')
CSV_FILE_VIDEO = os.path.join(CSV_FOLDER, 'input', 'youtube_comments_video.csv')
# CSV_FILE_SPORT = os.path.join(CSV_FOLDER, 'input', 'youtube_comments_sport.csv')
CSV_FILE_BETTERVIDEO = os.path.join(CSV_FOLDER, 'input', 'youtube_comments_bettervideo.csv')
CSV_FILE_PATH = os.path.join(CSV_FOLDER, 'youtube_comments.csv')
EMBEDDED_CSV_PATH = os.path.join(CSV_FOLDER, 'youtube_comments_embeded.csv')
CLUSTERED_CSV_PATH = os.path.join(CSV_FOLDER, 'youtube_comments_clustered.csv')
LABELED_CSV_PATH = os.path.join(CSV_FOLDER, 'youtube_comments_labeled.csv')
BATCH_FILE = "batch_input.jsonl"
OPENAI_MODEL = "text-embedding-3-large"
RATE_LIMIT_DELAY = 0.00  # in seconds

# --- OpenAI Client ---
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [7]:
def string_to_array(embedding_str):
    """Converts a string representation of an array to a NumPy array."""
    try:
        # Remove brackets and split by comma
        cleaned_str = embedding_str.strip('[]')
        values = cleaned_str.split(',')
        # Convert each value to float
        return np.array([float(val.strip()) for val in values])
    except:
        return None  # Handle potential errors in parsing

In [8]:
# Categories used for the labeling
category_dict = {
    "Vocal Performance": 1,

    "Visual Performance": 2,
    "Visual Performance - Stage Presence": 2.1,
    "Visual Performance - Gestures": 2.2,
    "Visual Performance - Movement": 2.3,

    "Technical Production": 3,
    "Technical Production - Video Quality": 3.1,
    "Technical Production - Camera Work": 3.2,
    "Technical Production - Lighting": 3.3,

    "Overall Impression": 4,
    "Overall Impression - Positive": 4.1,
    "Overall Impression - Negative": 4.2,
    "Overall Impression - Neutral": 4.3,

    "Engagement/Connection": 5,
    "Engagement/Connection - Personal Connection": 5.1,
    "Engagement/Connection - Artist Connection": 5.2,

    "Olympic games": 6,

    "Implicit Visual Influence": 7,

    "Playback usage": 8
}

# Preprocess input

In [9]:
import csv

In [10]:
df_music = pd.read_csv(CSV_FILE_MUSIC, sep='\t', quoting=csv.QUOTE_ALL, quotechar='"', encoding='utf-8')
df_video = pd.read_csv(CSV_FILE_VIDEO, sep='\t', quoting=csv.QUOTE_ALL, quotechar='"', encoding='utf-8')
#df_sport = pd.read_csv(CSV_FILE_SPORT, sep='\t', quoting=csv.QUOTE_ALL, quotechar='"', encoding='utf-8')
df_bettervideo = pd.read_csv(CSV_FILE_BETTERVIDEO, sep='\t', quoting=csv.QUOTE_ALL, quotechar='"', encoding='utf-8')

In [11]:
df_music['source_id'] = 1
df_video['source_id'] = 2
# df_sport['source_id'] = 2
df_bettervideo['source_id'] = 3

In [12]:
df = pd.concat([df_music, df_video, df_bettervideo], ignore_index=True) #add df_sport if needed

In [13]:
pd.DataFrame.to_csv(df, CSV_FILE_PATH)
del df_music
del df_video
# del df_sport
del df_bettervideo

# Labeling

In [14]:
df = pd.read_csv(CSV_FILE_PATH)

/var/folders/b5/hxjnqzqj32l4jyk_bkplnc9w0000gn/T/ipykernel_2915/3726769479.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_FILE_PATH)


In [15]:
system_message = """You are a comment classifier. Your task is to categorize comments based on a predefined set of categories and return their numerical IDs as a Python list. You must ONLY output a list of integers, and nothing else. Do not include any category names or other text in your response."""

user_template = """
Here are the categories and their IDs:
{category_dict}

Please read the comment below and identify all relevant categories.  Return ONLY the numerical IDs of the relevant categories as a Python list.

**Instructions:**

* **Identify Relevant Categories:** Determine which categories from the list above apply to the comment.
* **Output Format:** Return ONLY a Python list containing the numerical IDs of the relevant categories. The list should look like this: `[ID1, ID2, ID3]` (e.g., `[1, 8]` or `[3]` or `[]`).
* **Multiple Categories:** If multiple categories are relevant, include all their IDs in the list, separated by commas within the brackets.
* **No Relevant Categories:** If no categories are relevant, return an empty list `[]`.
* **Numerical IDs ONLY:**  Do NOT include category names or any other text in your response. Just the list of numerical IDs.

**Example Outputs:**
* For a comment belonging to categories 1 and 8: `[1, 8]`
* For a comment belonging to category 3: `[3]`
* For a comment belonging to no categories: `[]`

**Comment Text:**
{comment_text}

Category(ies) ID:
"""

In [ ]:
def create_batch_input_files(df, category_dict, user_template, max_comments_per_batch=15000):
    """Creates multiple JSONL files for Batch API input with comment limit per batch."""
    total_rows = len(df)
    batch_files = []
    
    # Calculate number of batches needed based on max comments per batch
    num_batches = (total_rows + max_comments_per_batch - 1) // max_comments_per_batch
    
    for batch_num in range(num_batches):
        start_idx = batch_num * max_comments_per_batch
        end_idx = min((batch_num + 1) * max_comments_per_batch, total_rows)
        batch_filename = f"batch_input_{batch_num + 1}.jsonl"
        
        # Get batch subset of DataFrame
        batch_df = df.iloc[start_idx:end_idx]
        
        print(f"Creating batch {batch_num + 1}/{num_batches} with {len(batch_df)} comments")
        
        with open(batch_filename, 'w') as f:
            for index, row in batch_df.iterrows():
                user_message = user_template.format(
                    comment_text=row['comment_text'], 
                    category_dict=json.dumps(category_dict)
                )
                request_data = {
                    "custom_id": str(index),
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": "gpt-4o-mini",
                        "messages": [
                            {"role": "system", "content": system_message},
                            {"role": "user", "content": user_message}
                        ],
                        "temperature": 0.3,
                        "seed": 42
                    }
                }
                f.write(json.dumps(request_data) + '\n')
        
        batch_files.append(batch_filename)
        print(f"Created batch file: {batch_filename}")
    
    return batch_files

def process_batch_results(output_file_path):
    """Process the batch results from the output file and return as a dictionary."""
    results = {}
    
    with open(output_file_path, 'r') as f:
        for line in f:
            response_data = json.loads(line)
            custom_id = response_data.get('custom_id')
            
            try:
                # Navigate through the response structure
                if ('response' in response_data and 
                    'body' in response_data['response']):
                    
                    # Parse the body as JSON
                    body_str = response_data['response']['body']
                    if isinstance(body_str, str):
                        body = json.loads(body_str)
                    else:
                        body = body_str
                    
                    # Extract the content from the assistant's message
                    if ('choices' in body and 
                        len(body['choices']) > 0 and 
                        'message' in body['choices'][0] and
                        'content' in body['choices'][0]['message']):
                        
                        content = body['choices'][0]['message']['content']
                        
                        # Parse the content as a JSON array
                        try:
                            labels = json.loads(content)
                            results[custom_id] = labels
                        except json.JSONDecodeError:
                            print(f"Warning: Could not parse content as JSON for ID {custom_id}: {content}")
                            # If parsing fails, store the raw content
                            results[custom_id] = content
            except Exception as e:
                print(f"Error processing result for ID {custom_id}: {str(e)}")
    
    print(f"Processed {len(results)} results from batch")
    return results

def process_batch_with_retries(client, batch_file, max_retries=3, retry_delay=60):
    """Process a single batch file with retries and better error handling."""
    print(f"Processing batch file: {batch_file}")
    
    for attempt in range(max_retries):
        try:
            # Upload batch file
            with open(batch_file, "rb") as f:
                response = client.files.create(file=f, purpose="batch")
            input_file_id = response.id
            
            # Create batch
            batch_response = client.batches.create(
                input_file_id=input_file_id,
                endpoint="/v1/chat/completions",
                completion_window="24h"
            )
            batch_id = batch_response.id
            print(f"Batch created with ID: {batch_id}")
            
            # Monitor batch status
            while True:
                batch_status = client.batches.retrieve(batch_id)
                print(f"Batch status: {batch_status.status}")
                
                if batch_status.status == "completed":
                    output_file_id = batch_status.output_file_id
                    output_file_response = client.files.content(output_file_id)
                    output_file_path = f"batch_output_{batch_id}.jsonl"
                    
                    with open(output_file_path, 'w') as f:
                        f.write(output_file_response.text)
                    
                    return process_batch_results(output_file_path)
                
                elif batch_status.status in ["failed", "expired", "cancelled"]:
                    print(f"Batch failed with status: {batch_status.status}")
                    if batch_status.error_file_id:
                        error_file_response = client.files.content(batch_status.error_file_id)
                        print("Error details:", error_file_response.text)
                    break
                
                time.sleep(60)  # Check every minute
                
        except Exception as e:
            print(f"Attempt {attempt + 1} failed with error: {str(e)}")
            if attempt < max_retries - 1:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print("Max retries reached. Moving to next batch.")
                return {}
    
    return {}

def process_all_batches(df, category_dict, user_template, client):
    """Process all batches with progress tracking and error handling."""
    print("Starting batch processing...")
    
    # Create batch files
    batch_files = create_batch_input_files(df, category_dict, user_template)
    all_results = {}
    processed_batches = 0
    total_batches = len(batch_files)
    
    # Process each batch
    for batch_file in batch_files:
        processed_batches += 1
        print(f"\nProcessing batch {processed_batches}/{total_batches}")
        
        batch_results = process_batch_with_retries(client, batch_file)
        all_results.update(batch_results)
        
        # Save intermediate results
        intermediate_df = df.copy()
        intermediate_df['comment_labels'] = intermediate_df.index.map(lambda x: all_results.get(str(x), []))
        intermediate_df.to_csv(f"intermediate_results_{processed_batches}.csv")
        print(f"Saved intermediate results to: intermediate_results_{processed_batches}.csv")
    
    return all_results

In [2]:
#batch_results = process_batch_results('/data/notebook_files/projfiles/input/batch_67b701252b4c8190b77ec9e319c0b008_output.jsonl')

all_results = process_all_batches(df, category_dict, user_template, client)

# Update DataFrame with final results
df['comment_labels'] = df.index.map(lambda x: all_results.get(str(x), []))

NameError: name 'df' is not defined

In [ ]:
pd.DataFrame.to_csv(df, LABELED_CSV_PATH)
print("Labeling completed and saved to:", LABELED_CSV_PATH)

# tags_process

In [ ]:
df = pd.read_csv(LABELED_CSV_PATH)
import ast  # import ast module for literal evaluation of strings

In [ ]:
def add_category_indicators(df, category_dict):
    """
    Adds binary indicator columns (1/0) for each category to the input DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame with 'comment_labels' column
        category_dict (dict): Dictionary defining categories and their codes

    Returns:
        pd.DataFrame: Original DataFrame with additional binary category columns
    """
    # Create new columns for each category, initialized with 0
    for category_name in category_dict.keys():
        df[f'has_{category_name}'] = 0
    
    # Process each row
    for idx, row in df.iterrows():
        labels_str = row['comment_labels']
        labels_list = []
        
        # Parse the labels
        if isinstance(labels_str, str):
            try:
                labels_list = ast.literal_eval(labels_str)
                if not isinstance(labels_list, list):
                    labels_list = []
            except (ValueError, SyntaxError):
                print(f"Warning: Could not parse comment_labels string: '{labels_str}'. Treating as no labels.")
                labels_list = []
        elif isinstance(labels_str, list):
            labels_list = labels_str
            
        # Set indicators for present categories
        if isinstance(labels_list, list):
            for label in labels_list:
                for cat_name, cat_code in category_dict.items():
                    if label == cat_code:
                        df.at[idx, f'has_{cat_name}'] = 1
                        
    return df

In [ ]:
# Add binary category indicators
df = add_category_indicators(df, category_dict)

# Kruskal–Wallis test

In [ ]:
from scipy.stats import kruskal
from ast import literal_eval  # converting strings to lists
from collections import defaultdict

In [ ]:
#df = pd.read_csv(LABELED_CSV_PATH)

In [ ]:
from scipy.stats import kruskal
from ast import literal_eval
from collections import defaultdict

def perform_kruskal_wallis_test(df, category_dict):
    """
    Performs Kruskal-Wallis H-test for each category across different video sources.
    
    Args:
        df (pd.DataFrame): DataFrame with 'source_id' and binary category columns (has_*)
        category_dict (dict): Dictionary defining categories and their codes
    """
    kruskal_results = {}
    
    # Test each category
    for category_name in category_dict.keys():
        column_name = f'has_{category_name}'
        if column_name not in df.columns:
            continue
            
        # Group data by source_id
        category_presence_by_source = defaultdict(list)
        for source in sorted(df['source_id'].unique()):
            source_data = df[df['source_id'] == source][column_name].tolist()
            category_presence_by_source[source] = source_data
            
        # Perform Kruskal-Wallis test
        try:
            h_statistic, p_value = kruskal(*[category_presence_by_source[s] for s in sorted(df['source_id'].unique())])
            kruskal_results[category_name] = {
                'H_statistic': h_statistic, 
                'p_value': p_value, 
                'groups_data': category_presence_by_source
            }
        except ValueError as e:
            kruskal_results[category_name] = {
                'H_statistic': np.nan, 
                'p_value': np.nan, 
                'error': str(e), 
                'groups_data': category_presence_by_source
            }
            
    # Print results
    print("Kruskal-Wallis Test Results for Category Distribution across Video Types:\n")
    for category, result in kruskal_results.items():
        print(f"Category: {category}")
        if 'error' in result:
            print(f"  Error during Kruskal-Wallis test: {result['error']}")
        else:
            print(f"  Kruskal-Wallis H-statistic: {result['H_statistic']:.4f}")
            print(f"  P-value: {result['p_value']:.4f}")
            
            if result['p_value'] < 0.05:
                print(f"  **Statistically Significant (p < 0.05)**: Reject Null Hypothesis. "
                      f"Distribution of '{category}' comments differs across video types.")
            else:
                print(f"  Not Statistically Significant (p >= 0.05): Fail to Reject Null Hypothesis. "
                      f"No strong evidence that distribution of '{category}' comments differs across video types.")
                
            print("\n  Category Presence Indicators by Source:")
            for source_id, data in result['groups_data'].items():
                print(f"    Source {source_id}:  Present Count = {sum(data)}, "
                      f"Absent Count = {len(data) - sum(data)}, Total = {len(data)}")
            print("-" * 50)
            
    return kruskal_results

In [ ]:
results = perform_kruskal_wallis_test(df, category_dict)

Kruskal-Wallis Test Results for Category Distribution across Video Types:

Category: Vocal Performance
  Kruskal-Wallis H-statistic: 1.4940
  P-value: 0.4738
  Not Statistically Significant (p >= 0.05): Fail to Reject Null Hypothesis. No strong evidence that distribution of 'Vocal Performance' comments differs across video types.

  Category Presence Indicators by Source:
    Source 1:  Present Count = 45, Absent Count = 196, Total = 241
    Source 2:  Present Count = 1861, Absent Count = 9923, Total = 11784
    Source 3:  Present Count = 416, Absent Count = 2187, Total = 2603
--------------------------------------------------
Category: Visual Performance
  Kruskal-Wallis H-statistic: 1.3686
  P-value: 0.5044
  Not Statistically Significant (p >= 0.05): Fail to Reject Null Hypothesis. No strong evidence that distribution of 'Visual Performance' comments differs across video types.

  Category Presence Indicators by Source:
    Source 1:  Present Count = 2, Absent Count = 239, Total = 2